In [2]:
# !pip install  dgl==2.0.0 -f https://data.dgl.ai/wheels/repo.html
# !pip install  dglgo==0.0.2 -f https://data.dgl.ai/wheels-test/repo.html

!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu121/repo.html

!pip install ogb

Looking in links: https://data.dgl.ai/wheels/torch-2.4/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━

In [3]:
import time

import dgl
import dgl.nn.pytorch as dglnn

import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
from ogb.nodeproppred import DglNodePropPredDataset

import gc

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.


# Загрузка датасета

In [4]:
device = th.device("cuda")
device

device(type='cuda')

In [5]:
data = DglNodePropPredDataset(name="ogbn-products")
splitted_idx = data.get_idx_split()
train_idx, val_idx, test_idx = (
    splitted_idx["train"],
    splitted_idx["valid"],
    splitted_idx["test"],
)
graph, labels = data[0]
nfeat = graph.ndata.pop("feat").to(device)
labels = labels[:, 0].to(device)

in_feats = nfeat.shape[1]
n_classes = (labels.max() + 1).item()
# Create csr/coo/csc formats before launching sampling processes
# This avoids creating certain formats in each data loader process, which saves momory and CPU.
graph.create_formats_()
# Pack data
data = (
    train_idx,
    val_idx,
    test_idx,
    in_feats,
    labels,
    n_classes,
    nfeat,
    graph,
)

This will download 1.38GB. Will you proceed? (y/N)
 y


Downloaded 1.38 GB: 100%|██████████| 1414/1414 [00:40<00:00, 34.91it/s]


Extracting dataset/products.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00,  2.71it/s]


Saving...


# Model

In [6]:
BATCH_SIZE = 1000
NUM_WORKERS = 4
NUM_HIDDEN = 256
NUM_LAYERS = 3
DROPOUT = 0.5
NUM_EPOCHS = 10
EVAL_EVERY = 1
LOG_EVERY = 20
SAVE_PRED = ""
FAN_OUT = "5,10,15"
LR = 0.003
WD = 0

In [7]:
class SAGE(nn.Module):
    def __init__(
        self, in_feats, n_hidden, n_classes, n_layers, activation, dropout
    ):
        super().__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.layers = nn.ModuleList()
        self.layers.append(dglnn.SAGEConv(in_feats, n_hidden, "mean"))
        for i in range(1, n_layers - 1):
            self.layers.append(dglnn.SAGEConv(n_hidden, n_hidden, "mean"))
        self.layers.append(dglnn.SAGEConv(n_hidden, n_classes, "mean"))
        self.dropout = nn.Dropout(dropout)
        self.activation = activation

    def forward(self, blocks, x):
        h = x
        for l, (layer, block) in enumerate(zip(self.layers, blocks)):
            # We need to first copy the representation of nodes on the RHS from the
            # appropriate nodes on the LHS.
            # Note that the shape of h is (num_nodes_LHS, D) and the shape of h_dst
            # would be (num_nodes_RHS, D)
            h_dst = h[: block.num_dst_nodes()]
            # Then we compute the updated representation on the RHS.
            # The shape of h now becomes (num_nodes_RHS, D)
            h = layer(block, (h, h_dst))
            if l != len(self.layers) - 1:
                h = self.activation(h)
                h = self.dropout(h)
        return h

    def inference(self, g, x, device):
        """
        Inference with the GraphSAGE model on full neighbors (i.e. without neighbor sampling).
        g : the entire graph.
        x : the input of entire node set.
        The inference code is written in a fashion that it could handle any number of nodes and
        layers.
        """
        # During inference with sampling, multi-layer blocks are very inefficient because
        # lots of computations in the first few layers are repeated.
        # Therefore, we compute the representation of all nodes layer by layer.  The nodes
        # on each layer are of course splitted in batches.
        # TODO: can we standardize this?
        for l, layer in enumerate(self.layers):
            y = th.zeros(
                g.num_nodes(),
                self.n_hidden if l != len(self.layers) - 1 else self.n_classes,
            ).to(device)

            sampler = dgl.dataloading.MultiLayerFullNeighborSampler(1)
            dataloader = dgl.dataloading.DataLoader(
                g,
                th.arange(g.num_nodes()),
                sampler,
                batch_size=BATCH_SIZE,
                shuffle=True,
                drop_last=False,
                num_workers=NUM_WORKERS,
            )

            for input_nodes, output_nodes, blocks in tqdm.tqdm(dataloader):
                block = blocks[0].int().to(device)

                h = x[input_nodes]
                h_dst = h[: block.num_dst_nodes()]
                h = layer(block, (h, h_dst))
                if l != len(self.layers) - 1:
                    h = self.activation(h)
                    h = self.dropout(h)

                y[output_nodes] = h

            x = y
        return y

In [8]:
def compute_acc(pred, labels):
    """
    Compute the accuracy of prediction given the labels.
    """
    return (th.argmax(pred, dim=1) == labels).float().sum() / len(pred)


def evaluate(model, g, nfeat, labels, val_nid, test_nid, device):
    """
    Evaluate the model on the validation set specified by ``val_mask``.
    g : The entire graph.
    inputs : The features of all the nodes.
    labels : The labels of all the nodes.
    val_mask : A 0-1 mask indicating which nodes do we actually compute the accuracy for.
    device : The GPU device to evaluate on.
    """
    model.eval()
    with th.no_grad():
        pred = model.inference(g, nfeat, device)
    model.train()
    return (
        compute_acc(pred[val_nid], labels[val_nid]),
        compute_acc(pred[test_nid], labels[test_nid]),
        pred,
    )

def load_subtensor(nfeat, labels, seeds, input_nodes):
    """
    Extracts features and labels for a set of nodes.
    """
    batch_inputs = nfeat[input_nodes]
    batch_labels = labels[seeds]
    return batch_inputs, batch_labels

# Train

In [9]:
train_nid, val_nid, test_nid, in_feats, labels, n_classes, nfeat, g = data

# Create PyTorch DataLoader for constructing blocks
sampler = dgl.dataloading.MultiLayerNeighborSampler(
    [int(fanout) for fanout in FAN_OUT.split(",")]
)
dataloader = dgl.dataloading.DataLoader(
    g,
    train_nid,
    sampler,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=False,
    num_workers=NUM_WORKERS,
)

In [10]:
model = SAGE(
    in_feats,
    NUM_HIDDEN,
    n_classes,
    NUM_LAYERS,
    F.relu,
    DROPOUT,
)
model = model.to(device)
loss_fcn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

In [11]:
avg = 0
iter_tput = []
best_eval_acc = 0
best_test_acc = 0
for epoch in range(NUM_EPOCHS):
    tic = time.time()

    # Loop over the dataloader to sample the computation dependency graph as a list of
    # blocks.
    for step, (input_nodes, seeds, blocks) in enumerate(dataloader):
        tic_step = time.time()

        # copy block to gpu
        blocks = [blk.int().to(device) for blk in blocks]

        # Load the input features as well as output labels
        batch_inputs, batch_labels = load_subtensor(
            nfeat, labels, seeds, input_nodes
        )

        # Compute loss and prediction
        batch_pred = model(blocks, batch_inputs)
        loss = loss_fcn(batch_pred, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        iter_tput.append(len(seeds) / (time.time() - tic_step))
        if step % LOG_EVERY == 0:
            acc = compute_acc(batch_pred, batch_labels)
            gpu_mem_alloc = (
                th.cuda.max_memory_allocated() / 1000000
                if th.cuda.is_available()
                else 0
            )
            print(
                "Epoch {:05d} | Step {:05d} | Loss {:.4f} | Train Acc {:.4f} | Speed (samples/sec) {:.4f} | GPU {:.1f} MB".format(
                    epoch,
                    step,
                    loss.item(),
                    acc.item(),
                    np.mean(iter_tput[3:]),
                    gpu_mem_alloc,
                )
            )
        gc.collect()

    toc = time.time()
    print("Epoch Time(s): {:.4f}".format(toc - tic))
    if epoch >= 5:
        avg += toc - tic
    if epoch % EVAL_EVERY == 0 and epoch != 0:
        eval_acc, test_acc, pred = evaluate(
            model, g, nfeat, labels, val_nid, test_nid, device
        )
        if SAVE_PRED:
            np.savetxt(
                SAVE_PRED + "%02d" % epoch,
                pred.argmax(1).cpu().numpy(),
                "%d",
            )
        print("Eval Acc {:.4f}".format(eval_acc))
        if eval_acc > best_eval_acc:
            best_eval_acc = eval_acc
            best_test_acc = test_acc
        print(
            "Best Eval Acc {:.4f} Test Acc {:.4f}".format(
                best_eval_acc, best_test_acc
            )
        )
    gc.collect()

print("Avg epoch time: {}".format(avg / (epoch - 4)))

/usr/local/lib/python3.10/dist-packages/dgl/dataloading/dataloader.py:1144: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Step 00000 | Loss 7.7536 | Train Acc 0.0280 | Speed (samples/sec) nan | GPU 1707.1 MB
Epoch 00000 | Step 00020 | Loss 1.5385 | Train Acc 0.6820 | Speed (samples/sec) 54367.3826 | GPU 1715.9 MB
Epoch 00000 | Step 00040 | Loss 0.8794 | Train Acc 0.7780 | Speed (samples/sec) 54763.7308 | GPU 1716.2 MB
Epoch 00000 | Step 00060 | Loss 0.7643 | Train Acc 0.8150 | Speed (samples/sec) 55251.6711 | GPU 1717.2 MB
Epoch 00000 | Step 00080 | Loss 0.5721 | Train Acc 0.8480 | Speed (samples/sec) 56242.9250 | GPU 1717.2 MB
Epoch 00000 | Step 00100 | Loss 0.6614 | Train Acc 0.8380 | Speed (samples/sec) 56602.8436 | GPU 1717.2 MB
Epoch 00000 | Step 00120 | Loss 0.6695 | Train Acc 0.8240 | Speed (samples/sec) 56811.7797 | GPU 1717.2 MB
Epoch 00000 | Step 00140 | Loss 0.5424 | Train Acc 0.8600 | Speed (samples/sec) 56841.8892 | GPU 1717.2 MB
Epoch 00000 | Step 00160 | Loss 0.5339 | Train Acc 0.8610 | Speed (samples/sec) 56401.1302 | GPU 1717.2 MB
Epoch 00000 | Step 00180 | Loss 0.4671 | Tra

100%|██████████| 2450/2450 [00:19<00:00, 125.81it/s]


Eval Acc 0.8960
Best Eval Acc 0.8960 Test Acc 0.7515
Epoch 00002 | Step 00000 | Loss 0.3342 | Train Acc 0.9060 | Speed (samples/sec) 52780.7229 | GPU 6345.1 MB
Epoch 00002 | Step 00020 | Loss 0.4045 | Train Acc 0.8920 | Speed (samples/sec) 52847.3061 | GPU 6345.1 MB
Epoch 00002 | Step 00040 | Loss 0.4167 | Train Acc 0.8860 | Speed (samples/sec) 52822.1809 | GPU 6345.1 MB
Epoch 00002 | Step 00060 | Loss 0.3190 | Train Acc 0.8970 | Speed (samples/sec) 52841.8415 | GPU 6345.1 MB
Epoch 00002 | Step 00080 | Loss 0.4317 | Train Acc 0.8840 | Speed (samples/sec) 52988.0429 | GPU 6345.1 MB
Epoch 00002 | Step 00100 | Loss 0.3953 | Train Acc 0.9050 | Speed (samples/sec) 52888.8023 | GPU 6345.1 MB
Epoch 00002 | Step 00120 | Loss 0.3754 | Train Acc 0.8910 | Speed (samples/sec) 53217.1732 | GPU 6345.1 MB
Epoch 00002 | Step 00140 | Loss 0.3351 | Train Acc 0.9010 | Speed (samples/sec) 53536.7669 | GPU 6345.1 MB
Epoch 00002 | Step 00160 | Loss 0.3869 | Train Acc 0.8930 | Speed (samples/sec) 53625.0842 

100%|██████████| 2450/2450 [00:18<00:00, 129.74it/s]


...

Eval Acc 0.9141
Best Eval Acc 0.9141 Test Acc 0.7871
Epoch 00008 | Step 00000 | Loss 0.2886 | Train Acc 0.9200 | Speed (samples/sec) 52081.9199 | GPU 6813.0 MB
Epoch 00008 | Step 00020 | Loss 0.3072 | Train Acc 0.9080 | Speed (samples/sec) 52078.0135 | GPU 6813.0 MB
Epoch 00008 | Step 00040 | Loss 0.3149 | Train Acc 0.9100 | Speed (samples/sec) 52064.6879 | GPU 6813.0 MB
Epoch 00008 | Step 00060 | Loss 0.3358 | Train Acc 0.9070 | Speed (samples/sec) 52106.6597 | GPU 6813.0 MB
Epoch 00008 | Step 00080 | Loss 0.3293 | Train Acc 0.9040 | Speed (samples/sec) 52022.2845 | GPU 6813.0 MB
Epoch 00008 | Step 00100 | Loss 0.2550 | Train Acc 0.9320 | Speed (samples/sec) 52000.0891 | GPU 6813.0 MB
Epoch 00008 | Step 00120 | Loss 0.3787 | Train Acc 0.8930 | Speed (samples/sec) 51956.3177 | GPU 6813.0 MB
Epoch 00008 | Step 00140 | Loss 0.3461 | Train Acc 0.9080 | Speed (samples/sec) 51921.6744 | GPU 6813.0 MB
Epoch 00008 | Step 00160 | Loss 0.3253 | Train Acc 0.9070 | Speed (samples/sec) 51860.9294 

100%|██████████| 2450/2450 [00:18<00:00, 134.64it/s]


Eval Acc 0.9157
Best Eval Acc 0.9157 Test Acc 0.7907
Epoch 00009 | Step 00000 | Loss 0.3104 | Train Acc 0.9030 | Speed (samples/sec) 51817.2121 | GPU 6813.0 MB
Epoch 00009 | Step 00020 | Loss 0.3491 | Train Acc 0.9060 | Speed (samples/sec) 51796.6883 | GPU 6813.0 MB
Epoch 00009 | Step 00040 | Loss 0.2841 | Train Acc 0.9140 | Speed (samples/sec) 51777.2843 | GPU 6813.0 MB
Epoch 00009 | Step 00060 | Loss 0.2798 | Train Acc 0.9200 | Speed (samples/sec) 51703.0830 | GPU 6813.0 MB
Epoch 00009 | Step 00080 | Loss 0.2531 | Train Acc 0.9250 | Speed (samples/sec) 51635.9527 | GPU 6813.0 MB
Epoch 00009 | Step 00100 | Loss 0.2967 | Train Acc 0.9180 | Speed (samples/sec) 51658.3196 | GPU 6813.0 MB
Epoch 00009 | Step 00120 | Loss 0.2819 | Train Acc 0.9110 | Speed (samples/sec) 51617.8213 | GPU 6813.0 MB
Epoch 00009 | Step 00140 | Loss 0.3481 | Train Acc 0.8990 | Speed (samples/sec) 51632.3964 | GPU 6813.0 MB
Epoch 00009 | Step 00160 | Loss 0.3124 | Train Acc 0.9090 | Speed (samples/sec) 51634.1312 

100%|██████████| 2450/2450 [00:18<00:00, 134.48it/s]


Eval Acc 0.9171
Best Eval Acc 0.9171 Test Acc 0.7901
Avg epoch time: 42.135081911087035


In [13]:
import torch
torch.save(model, './SAGE.pt')